# IME672A GROUP-5 <br> Medical Insurance Cost Prediction
## Part 2 - Predicting data

#### Installing extra libraries 

In [75]:
!pip install xgboost

#### Importing the requie 

In [76]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import time

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import xgboost as xg
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

In [77]:
df=pd.read_csv(r"D:\ime672\project ime672\archive\insurance.csv")
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [78]:
# Find the datatype and count of each column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [79]:
df.isna().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [80]:
df["smoker"] = df["smoker"].astype("category")
df = pd.get_dummies(df, columns= ["smoker"])

In [81]:
df["sex"] = df["sex"].astype("category")
df = pd.get_dummies(df, columns= ["sex"])

In [82]:
df["region"] = df["region"].astype("category")
df = pd.get_dummies(df, columns= ["region"])

In [83]:
df.head()

,age,bmi,children,charges,smoker_no,smoker_yes,sex_female,sex_male,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,0,1,1,0,0,0,0,1
1,18,33.770,1,1725.55230,1,0,0,1,0,0,1,0
2,28,33.000,3,4449.46200,1,0,0,1,0,0,1,0
3,33,22.705,0,21984.47061,1,0,0,1,0,1,0,0
4,32,28.880,0,3866.85520,1,0,0,1,0,1,0,0


In [84]:
scaler=MinMaxScaler()
# transform data
d1=scaler.fit_transform(df)
data1=pd.DataFrame(d1)
data1

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.021739,0.321227,0.0,0.251611,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.000000,0.479150,0.2,0.009636,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.217391,0.458434,0.6,0.053115,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.326087,0.181464,0.0,0.333010,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.304348,0.347592,0.0,0.043816,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1333,0.695652,0.403820,0.6,0.151299,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1334,0.000000,0.429379,0.0,0.017305,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1335,0.000000,0.562012,0.0,0.008108,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1336,0.065217,0.264730,0.0,0.014144,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [85]:
mx=list(df.columns)

In [86]:
data1.columns=mx

In [87]:
data1

,age,bmi,children,charges,smoker_no,smoker_yes,sex_female,sex_male,region_northeast,region_northwest,region_southeast,region_southwest
0,0.021739,0.321227,0.0,0.251611,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.000000,0.479150,0.2,0.009636,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.217391,0.458434,0.6,0.053115,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.326087,0.181464,0.0,0.333010,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.304348,0.347592,0.0,0.043816,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1333,0.695652,0.403820,0.6,0.151299,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1334,0.000000,0.429379,0.0,0.017305,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1335,0.000000,0.562012,0.0,0.008108,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1336,0.065217,0.264730,0.0,0.014144,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [88]:
x=data1.drop('charges',axis=1)
y=data1['charges']

In [89]:
test_size=0.2
x_train,x_test,y_train,y_test = train_test_split(x, y,test_size=0.2,random_state=20)

In [90]:
model1=LinearRegression()
model1.fit(x_train,y_train)
model1.score(x_test,y_test)

0.7959688488543413

In [91]:
model2=RandomForestRegressor(random_state=0)
model2.fit(x_train,y_train)
model2.score(x_test,y_test)

0.8902677282217355

In [122]:
model3 = xg.XGBRegressor(objective = 'reg:squarederror', booster = 'gblinear',random_state=0)
model3.fit(x_train,y_train)
model3.score(x_test,y_test)

0.795963771364391

In [118]:
model3.predict(x_train)

array([0.27771407, 0.07855193, 0.00680086, ..., 0.08572646, 0.6632853 ,
       0.38808078], dtype=float32)

In [93]:
from sklearn.neighbors import KNeighborsRegressor
model4=KNeighborsRegressor(n_neighbors=5)
model4.fit(x_train,y_train)
model4.score(x_test,y_test)

0.8321870828363187

In [94]:
from sklearn.ensemble import GradientBoostingRegressor
model5=GradientBoostingRegressor(random_state=0)
model5.fit(x_train, y_train)
model5.score(x_test,y_test)

0.9042927364082448

In [95]:
from sklearn.ensemble import AdaBoostRegressor
model6=AdaBoostRegressor(random_state=0, n_estimators=100)
model6.fit(x_train, y_train)
model6.score(x_test,y_test)

0.855113325248656

## Best regressor is gradient boosting regressor

In [96]:
n_estimators = [100, 200]
learning_rate=[0.01,0.1]
min_samples_split = [1,5]
min_samples_leaf = [1,2]

In [97]:
random_grid={'n_estimators': n_estimators,'learning_rate': learning_rate,'min_samples_split': min_samples_split,'min_samples_leaf': min_samples_leaf}

In [98]:
gb_grid = RandomizedSearchCV(estimator=GradientBoostingRegressor(),param_distributions=random_grid,cv=3,verbose=2,n_jobs=4)

In [99]:
gb_grid.fit(x_train,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.82386978 0.82911356 0.82867042 0.72730224
 0.82068729        nan        nan 0.83743942]
  warnings.warn(


RandomizedSearchCV(cv=3, estimator=GradientBoostingRegressor(), n_jobs=4,
                   param_distributions={'learning_rate': [0.01, 0.1],
                                        'min_samples_leaf': [1, 2],
                                        'min_samples_split': [1, 5],
                                        'n_estimators': [100, 200]},
                   verbose=2)

In [100]:
print(gb_grid.best_score_)
print(gb_grid.best_params_)

0.837439422940563
{'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 2, 'learning_rate': 0.1}


In [101]:
from sklearn.ensemble import GradientBoostingRegressor
final=GradientBoostingRegressor(n_estimators=100,min_samples_split=5,min_samples_leaf=2,learning_rate=0.1,random_state=2)

In [102]:
final.fit(x_train, y_train)
acc=final.score(x_test,y_test)

In [103]:
print("Accuracy of the model is : ",acc*100,"%")

Accuracy of the model is :  90.71388613394112 %


In [104]:
ypred=final.predict(x_test)

In [105]:
ypred

array([0.17528357, 0.14197527, 0.14702649, 0.25071333, 0.05216966,
       0.05003032, 0.06593268, 0.14064376, 0.43261666, 0.14517638,
       0.26142267, 0.59226219, 0.08479378, 0.18694135, 0.25182066,
       0.14533281, 0.32940305, 0.14025439, 0.0482959 , 0.17587393,
       0.73974911, 0.13802782, 0.17569579, 0.04489799, 0.08978538,
       0.10742906, 0.37990753, 0.21480808, 0.56119867, 0.13558847,
       0.1303858 , 0.0690261 , 0.46399202, 0.7150296 , 0.22189509,
       0.04593742, 0.11324069, 0.05617217, 0.34898545, 0.22970528,
       0.20203163, 0.19747612, 0.02624441, 0.04481296, 0.26074537,
       0.12502857, 0.06336797, 0.10653336, 0.69709954, 0.06215867,
       0.12238579, 0.42700497, 0.14671837, 0.17528357, 0.0855595 ,
       0.53602801, 0.10576001, 0.01760264, 0.02490405, 0.30792457,
       0.546434  , 0.36501601, 0.20095393, 0.12059824, 0.1354711 ,
       0.1515202 , 0.15014303, 0.1111821 , 0.42181392, 0.61661016,
       0.15646115, 0.20639333, 0.21006437, 0.32419382, 0.14302

In [106]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse

In [107]:
abs_error = mae(ypred,y_test)
sq_error=mse(ypred,y_test)

In [108]:
print("Mean absolute error : ",abs_error)
print("Mean squared error : ",sq_error)

Mean absolute error :  0.03462482512422409
Mean squared error :  0.003557803479946421
